In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Part 2 - Fitting the CNN to the images

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [4]:
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000, 
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
8000/8000 [==============================] - 2775s 347ms/step - loss: 0.3679 - acc: 0.8275 - val_loss: 0.5555 - val_acc: 0.8167
Epoch 2/25
8000/8000 [==============================] - 2728s 341ms/step - loss: 0.1182 - acc: 0.9542 - val_loss: 1.0211 - val_acc: 0.7877
Epoch 3/25
8000/8000 [==============================] - 2731s 341ms/step - loss: 0.0575 - acc: 0.9794 - val_loss: 1.1847 - val_acc: 0.7939
Epoch 4/25
8000/8000 [==============================] - 2738s 342ms/step - loss: 0.0424 - acc: 0.9851 - val_loss: 1.2194 - val_acc: 0.7981
Epoch 5/25
8000/8000 [==============================] - 2741s 343ms/step - loss: 0.0329 - acc: 0.9885 - val_loss: 1.3715 - val_acc: 0.8061
Epoch 6/25
8000/8000 [==============================] - 2738s 342ms/step - loss: 0.0278 - acc: 0.9905 - val_loss: 1.3713 - val_acc: 0.7957
Epoch 7/25
8000/8000 [==============================] - 2743s 343ms/step - loss: 0.0243 - acc: 0.9918 - val_loss: 1.49

In [5]:
loss, acc = classifier.evaluate_generator(generator=test_set, steps=2000)

In [6]:
print(loss)

1.6109816778910997


In [7]:
print(acc)

0.8069241431451613
